In [18]:
import pandas as pd
import os
from dotenv import load_dotenv
from google.cloud.sql.connector import Connector, IPTypes
import pg8000
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# Loading environment variables
load_dotenv()

instance_connection_name = os.environ["INSTANCE_CONNECTION_NAME"]
db_user = os.environ["DB_USER"]
db_pass = os.environ["DB_PASS"]
db_name = os.environ["DB_NAME"]

ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

print(f"Connection to: {instance_connection_name}")
print(f"Database: {db_name}")
print(f"User: {db_user}")

Connection to: project-b9a39240-863d-4573-9f6:us-central1:free-trial-first-project
Database: postgres
User: postgres


In [20]:
# Creating a connection
connector = Connector(refresh_strategy="LAZY")

def get_conn() -> pg8000.dbapi.Connection:
    conn: pg8000.dbapi.Connection = connector.connect(
        instance_connection_name,
        "pg8000",
        user=db_user,
        password=db_pass,
        db=db_name,
        ip_type=ip_type,
    )
    return conn

conn = get_conn()
print("Connection established successfully!")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x12c4619a0>


Connection established successfully!


In [21]:
# Download all data from the table
query = """
    SELECT 
        id,
        vehicle_id,
        area,
        x,
        y,
        width,
        heigth,
        date_time,
        frame_id
    FROM traffic_data
    ORDER BY date_time DESC
"""

df = pd.read_sql(query, conn)
print(f"Loaded {len(df)} records")
df.head()

/var/folders/23/mph96hlx47vgcr8nq2qkpjfc0000gn/T/ipykernel_88449/370571232.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Loaded 44026 records


,id,vehicle_id,area,x,y,width,heigth,date_time,frame_id
0,44026,574,882.0,138,0,50.0,19.0,2025-11-29 14:50:10.898700,57242
1,44025,574,816.0,138,0,49.0,18.0,2025-11-29 14:50:10.845837,57241
2,44024,574,931.0,138,0,50.0,20.0,2025-11-29 14:50:10.797287,57240
3,44023,574,817.0,143,0,44.0,20.0,2025-11-29 14:50:10.743950,57239
4,44022,573,836.0,146,0,39.0,23.0,2025-11-29 14:50:10.636935,57237


In [22]:
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['hour'] = df['date_time'].dt.hour
df['unique_vehicle_id'] = round(df['vehicle_id'] + df['year']/10000 + df['month']/1000000 + df['day']/100000000 + df['hour']/10000000000, 10)
df.head()

,id,vehicle_id,area,x,y,width,heigth,date_time,frame_id,year,month,day,hour,unique_vehicle_id
0,44026,574,882.0,138,0,50.0,19.0,2025-11-29 14:50:10.898700,57242,2025,11,29,14,574.202511
1,44025,574,816.0,138,0,49.0,18.0,2025-11-29 14:50:10.845837,57241,2025,11,29,14,574.202511
2,44024,574,931.0,138,0,50.0,20.0,2025-11-29 14:50:10.797287,57240,2025,11,29,14,574.202511
3,44023,574,817.0,143,0,44.0,20.0,2025-11-29 14:50:10.743950,57239,2025,11,29,14,574.202511
4,44022,573,836.0,146,0,39.0,23.0,2025-11-29 14:50:10.636935,57237,2025,11,29,14,573.202511


In [23]:
df['size'] = df['width'] * df['heigth']
df.head()

,id,vehicle_id,area,x,y,width,heigth,date_time,frame_id,year,month,day,hour,unique_vehicle_id,size
0,44026,574,882.0,138,0,50.0,19.0,2025-11-29 14:50:10.898700,57242,2025,11,29,14,574.202511,950.0
1,44025,574,816.0,138,0,49.0,18.0,2025-11-29 14:50:10.845837,57241,2025,11,29,14,574.202511,882.0
2,44024,574,931.0,138,0,50.0,20.0,2025-11-29 14:50:10.797287,57240,2025,11,29,14,574.202511,1000.0
3,44023,574,817.0,143,0,44.0,20.0,2025-11-29 14:50:10.743950,57239,2025,11,29,14,574.202511,880.0
4,44022,573,836.0,146,0,39.0,23.0,2025-11-29 14:50:10.636935,57237,2025,11,29,14,573.202511,897.0


In [24]:
min_max_df = df.groupby('unique_vehicle_id')[['size', 'y']].agg(['max', 'min']).sort_index()

In [25]:
min_max_df['way_size'] = min_max_df['y']['max'] - min_max_df['y']['min']
min_max_df.columns = ['size_max', 'size_min', 'y_max', 'y_min', 'way_size']
min_max_df.head()

,size_max,size_min,y_max,y_min,way_size
unique_vehicle_id,,,,,
0.202511,2961.0,2961.0,0,0,0
0.202511,2556.0,935.0,244,84,160
1.202511,1000.0,880.0,0,0,0
1.202511,12408.0,903.0,269,0,269
2.202511,902.0,902.0,0,0,0


In [28]:
min_max_df['full_way'] = min_max_df['way_size'] > 240

In [41]:
min_max_df['day_period'] = min_max_df.index.map(lambda x: 'day' if 6 <= int((x* 10000000000)%100) < 18 else 'night')

In [42]:
min_max_df

,size_max,size_min,y_max,y_min,way_size,full_way,day_period
unique_vehicle_id,,,,,,,
0.202511,2961.0,2961.0,0,0,0,False,night
0.202511,2556.0,935.0,244,84,160,False,day
1.202511,1000.0,880.0,0,0,0,False,night
1.202511,12408.0,903.0,269,0,269,True,day
2.202511,902.0,902.0,0,0,0,False,day
...,...,...,...,...,...,...,...
2642.202511,2850.0,2438.0,38,18,20,False,night
2643.202511,2480.0,1008.0,196,158,38,False,night
2644.202511,3888.0,3328.0,94,68,26,False,night


In [27]:
# Close the connection to the database
# conn.close()
# connector.close()
# print("✓ Connection closed")